    %load_ext pidgin.shell
    
The `pidgin` shell augments the `IPython` shell in the notebook to permit Markdown forward [computational essays](http://blog.stephenwolfram.com/2017/11/what-is-a-computational-essay/).   It provides:

* Markdown input in code cells.
* Markdown block code statements and expressions are evaluated as a normal code cell.
* Inline code expressions are evaluated asynchronously.
* The Markdown code accepts `jinja2` templates too


In [2]:
try:
    import commonmark
    from commonmark import Parser
    from commonmark.render.renderer import Renderer
except:
    import CommonMark
    from CommonMark import Parser
    from CommonMark.render.renderer import Renderer
from textwrap import indent, dedent


In [3]:
from .markdown import renderer
import IPython
import types, sys, IPython, asyncio, trio, traceback, ipykernel, traitlets, nbconvert, jinja2, operator, toolz.curried as toolz, tokenize, io, textwrap
from promise import Promise

ip = IPython.get_ipython()


In [4]:
def expression_tokens(str): 
    try:
        return list(tokenize.generate_tokens(io.StringIO(str).readline))
    except: return []

In [5]:
def split_expression(str, *expressions):
    """Split an expression on the semi colons."""
    start = 0
    if str.startswith(';'): 
        """This complies with the special IPython ; syntax"""
        return str, 
    for id in toolz.pipe(
        str, expression_tokens, 
        toolz.filter(toolz.compose(
            ';'.__eq__, operator.attrgetter('string')
        )), 
        toolz.map(toolz.compose(
            toolz.second,
            operator.attrgetter('start')
        )),
        list
    ) + [len(str)]:
        expressions += str[start:id].strip(),
        start = id + 1
    return expressions

In [6]:
def requote(str, token='"""'):
    if token in str: token = "'''"
    return token+str+token

In [7]:
    environment = jinja2.Environment(enable_async=False)
    environment.globals.update(vars(__import__('builtins')))

In [9]:
    ip = get_ipython()

In [48]:
    class PidginShell(IPython.core.interactiveshell.InteractiveShell):
        markdown = traitlets.Bool(True)
        template = traitlets.Bool(True)
        environment = traitlets.Instance(
            jinja2.Environment, 
            default_value=environment)
        expressions = traitlets.Bool(True)
        tangle_expressions = traitlets.Bool(True)
        
        
        def run_cell(self, code, store_history=False, silent=False, shell_futures=True, **user_expressions):
            """Run cell separates the sync and async parts."""
            self._last_traceback = None
            
            # Separate block and inline code from the markdown source.
            if self.markdown and not code.lstrip().startswith('%%'): 
                source = renderer(code, user_expressions=user_expressions)
            else: source = code

            silent = silent or not source
            display = None

            if self.markdown:
                # Create a markdown display object tjat may be updated later.
                if code.strip() and code.splitlines()[0].strip():
                    if textwrap.dedent(code).strip() != source.strip():
                        display = IPython.display.display(IPython.display.Markdown(code), display_id=True)

            if not source:
                try:
                    if self.template:
                        code = self.environment.from_string(code).render(**self.user_ns)
                except BaseException as e: 
                    traceback.print_exc()
                    result = IPython.core.interactiveshell.InteractiveShell.run_cell(
                        self, '', store_history=False, silent=silent, shell_futures=shell_futures)                    
                else:
                    result = IPython.core.interactiveshell.InteractiveShell.run_cell(
                        self, requote(code), store_history=False, silent=silent, shell_futures=shell_futures)        
            else: 
                result = IPython.core.interactiveshell.InteractiveShell.run_cell(
                    self, source, store_history=store_history, silent=silent, shell_futures=shell_futures)
            
            trio.run(
                self.async_expressions, code, display, store_history, silent, shell_futures, user_expressions)
            
            return result

        def sync_user_expression(self, code):
            expressions = split_expression(code)
            
            results = IPython.core.interactiveshell.InteractiveShell.user_expressions(self, dict(
                zip(expressions, map(self.input_transformer_manager.transform_cell, expressions))))

            IPython.display.display(IPython.display.Markdown('''`>>> {}`'''.format(code.strip('`'))))

            for expression, result in results.items():
                expressions_ns = '`{}`'.format(expression)
                if result['status'] == 'error': 
                    error_result = self._format_user_obj('')
                    error_result['data']['text/plain'] = ''.join(''.join(result['traceback']))
                    result = error_result
                elif expressions_ns in self.user_ns:
                    display = self.user_ns[expressions_ns]
                    IPython.display.publish_display_data(result['data'], update=True, transient={'display_id': display.display_id})

            if result['status'] == 'ok':
                display = self.user_ns[
                    "`{}`".format(code.strip())
                ] = self.user_ns[expressions_ns] = IPython.display.DisplayHandle()
                IPython.display.publish_display_data(result['data'], transient={
                    'display_id': display.display_id})
                
            return result
        
        async def async_user_expression(self, code): return self.sync_user_expression(code)
        
        async def async_expressions(self, code, display, store_history=False, silent=False, shell_futures=True, user_expressions=None):
            async with trio.open_nursery() as nursery:
                if user_expressions:
                    IPython.display.display(IPython.display.Markdown('---'))
                    for key, expression in user_expressions.items():
                        nursery.start_soon(self.async_user_expression, expression)

                if display and self.template: 
                    nursery.start_soon(self.update_display_template, display, code)

        async def update_display_template(self, display, code):
            try:
                display.update(IPython.display.Markdown(
                    self.environment.from_string(code).render(**self.user_ns)))
            except BaseException as e: traceback.print_exc()

In [49]:
    def get_inline_expression(string, pos):
        try:
            line, offset = IPython.utils.tokenutil.line_at_cursor(string, pos)
        except tokenize.TokenError: return 
        inline = True
        while (offset < pos) and line:
            inline = not inline
            snippet, token, line = line.partition('`')
            offset += len(snippet) + len(token)
        return inline and snippet or None

In [55]:
    class PidginKernel(type(ip.kernel)):        
        def do_inspect(self, code, cursor_pos, detail_level=0):
            inspect = False
            if (cursor_pos-2)>0 and cursor_pos-1 <= len(code) :
                if code[cursor_pos-1] == '?':
                    inspect = True
                    if code[cursor_pos-2] == '?':
                        detail_level = 1

            if not inspect and '`' in code:
                inline = get_inline_expression(code, cursor_pos)
                if inline:
                    return dict(self.shell.sync_user_expression(inline), found=True)

            result = ipykernel.ipkernel.IPythonKernel.do_inspect(self, code, cursor_pos, detail_level)
            
            if not result['found']:
                result['found'] = True
                result['data'].update({
                    'text/markdown': code
                })
            return result

 

In [56]:
    original_methods = {}
    def load_ipython_extension(ip):
        global original_methods
        for method in (
            PidginShell.run_cell, PidginShell.async_expressions, 
            PidginShell.async_user_expression, PidginShell.update_display_template,
            PidginShell.sync_user_expression
        ): 
            object = getattr(ip, method.__name__, None)
            if object:
                original_methods[method.__name__] = original_methods.get(method.__name__, object)
            setattr(ip, method.__name__, types.MethodType(method, ip))

        for trait in (
            'markdown', 'template', 'expressions', 'tangle_expressions', 'environment'
        ): 
            ip.add_traits(**{trait: getattr(PidginShell,  trait)})
            setattr(ip, trait, getattr(PidginShell,  trait).default_value)
        ip.kernel.do_inspect = types.MethodType(PidginKernel.do_inspect, ip.kernel)

    def unload_ipython_extension(ip): 
        ip.__dict__.update(original_methods)
        ip.kernel.do_inspect = types.MethodType(ipykernel.ipkernel.IPythonKernel.do_inspect, ip.kernel)


In [57]:
    if __name__ == '__main__':
        #!ipython -m pytest -- shell.ipynb
        load_ipython_extension(get_ipython())